In [ ]:
import pandas as pd

# Convert raw data to a pandas data frame.
full = pd.DataFrame({pbi.T: pd.Series(dict({'V': pbi.V, 'E': pbi.E, 'W': pbi.W(), 'S': pbi.S()})) for pbi in backlog}).transpose()

# Show a table with what you believe the weightiest item is (even if it's too large to do).
display(full.sort_values(by='W', ascending=False))

# Plot (V, E) with a label on every point with the summary of the Task. You should be able to quickly see how many
# tasks or stories are small enough to start on (less than 4-8 hours). Hopefully you always have at least 3-4 stories
# that are small enough you can pick from. Over time you should be able to see what kind of V/E ratio you typically have
# on tasks you actually do.
nominal = pd.DataFrame({'V': full['V'].map(lambda v: v.nominal_value),
                        'E': full['E'].map(lambda e: e.nominal_value)})
   
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8,8]

# https://stackoverflow.com/a/26000515/622049
fig, ax = plt.subplots()
nominal.plot('E', 'V', kind='scatter', ax=ax)

for k, v in nominal.iterrows():
    ax.annotate(s=k, xy=(v.E, v.V),
                xytext=(10,-5), textcoords='offset points',
                family='sans-serif', fontsize=16, color='darkslategrey')

ax.grid()